
# Shadow Keep - Labyrinth Decoding

In [ ]:
#py
#Shadow Keep - Labyrinth Decoding
# bmd
# 2024-09-08
#
# Decoding the LAbyrinth Dungeon from 1992 Game Shadow Keep
# Really wanted to see everything in this dungeon, and its really slow to map by hand!
#
# Would like to print it here, work out the codes and then assemble into a picture!

# Get the Map Data
f = open("raw_as_ascii.txt", "r")
map_data = f.read()
print(map_data)
size = len(map_data)
print(size) # Labyrinth # Length is 11700 -> this is roughly 108 blocks by 108 - > Not a perfect square. ok that doesnt matter. Answer is going to be around this!



In [ ]:
#What are the factors of 11700? -> The map is going to be approximately square-ish.
from functools import reduce

#https://stackoverflow.com/questions/6800193/what-is-the-most-efficient-way-of-finding-all-the-factors-of-a-number-in-python
def factors(n):
    return set(reduce(
        list.__add__,
        ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

potential_dimensions = factors(size)
print('All Dimensions:')
print(potential_dimensions)
potential_dimensions_2 = []

for item in potential_dimensions:
    if item > 100 and item < 500:
        potential_dimensions_2.append(item)

print('\nThe likely dimensions are:')
print(potential_dimensions_2)

In [ ]:
# That bottom one of 117x100 is the sort of dimension that seems right.
# Now want to draw the data in lines of x Long, i.e. 117 rows of 100, or 100 rows of 117 long!

def split_string_by_n_chars(input_string, n):
    return [input_string[i:i+n] for i in range(0, len(input_string), n)]

n = 130  # Number of characters per line
output = split_string_by_n_chars(map_data, n)
for line in output: print(line)

In [ ]:
for n in potential_dimensions_2:
    m = 11700/n
    print(f'Printing {n} by {m}')

for n in potential_dimensions_2:
    m = int(11700/n)
    print(f'Printing {m} by {n}') 
    output = split_string_by_n_chars(map_data, m)
    for line in output: print(line)
    print('\n\n')

## This lead to me finding it the result is 90 by 130

❗❗❗ This is the only version with perfectly straight lines down. and makes sense. can even recognise parts of the maze!!❗❗❗ 
Unique Chars: {' ', 'y', 'G', 'b', 'a', '$', ':', 'c', 'l'}

- ' ' is empty walking space ❓
- G is the START & EXIT!!
- $ is a wall
- C is treasure!
- ':' is a scroll that says "dead end"
- b: ❓❓❓
- a: ❓❓❓
- y: ❓❓❓
- l ❓❓❓


In [ ]:
output = split_string_by_n_chars(map_data, 90)
square_data_array = []
square_data_array_2 = []
for line in output: 
    #print(line)
    square_data_array.append(line)

#Split up the data into indiviudla array elements (inc per line)
for line in output:
    line_data = []
    for char in line:
        line_data.append(char)
    square_data_array_2.append(line_data)


special_chars = {'y', 'G', 'b', 'a', ':', 'c', 'l'}

#PRINT DATA VISUALLY
for line in square_data_array:
    for char in line:
        if char in special_chars:
            print(f"\x1b[31m{char}\x1b[0m", end='')
        else:
            print(char, end='')
    print('')

In [ ]:
def unique_characters(s):
    return set(s)

# Example usage
string = "hello world"
unique_chars = unique_characters(map_data)
print(unique_chars)

In [ ]:
# Happy to decode the hex into a visual :)
# its a little hard to follow as the fonts arent perfectly square.

#Can I now export this as an image?? I.e. a Perfect square reference drawing?
# Would be nicely replicable for the other maps too!!

# Example usage:
ascii_art = [
    [' ', ' ', 'G'],
    ['$', '$', '$'],
    [' ', '$', '$']
]
print(ascii_art)

print(square_data_array)
print(square_data_array_2)

## Now Render as a single image! (This Worked!!! EPIC! -> Used chatgpt to speed this up. worked great!)

In [25]:
from PIL import Image
import numpy as np

def create_image_from_ascii(ascii_art, img_map, output_size=(1024, 1024), char_size=(50, 50)):
    """
    Create an image by replacing ASCII characters with corresponding images.
    
    ascii_art: 2D array of characters (ASCII art)
    img_map: Dictionary mapping characters to image file paths
    output_size: Size of the final image (width, height)
    char_size: Size for each character-replacing image (width, height)
    """
    # Calculate final image size based on character size and ASCII dimensions
    ascii_height = len(ascii_art)
    ascii_width = len(ascii_art[0]) if ascii_height > 0 else 0
    
    final_image_width = ascii_width * char_size[0]
    final_image_height = ascii_height * char_size[1]
    
    final_image = Image.new("RGB", (final_image_width, final_image_height))

    # Iterate through each character in the ASCII art and paste corresponding image
    for i, row in enumerate(ascii_art):
        for j, char in enumerate(row):
            if char in img_map:
                # Open the image corresponding to the current ASCII character
                img = Image.open(img_map[char])
                
                # Resize the image to the specified character size
                img = img.resize(char_size)
                
                # Calculate the position to paste the image
                pos_x = j * char_size[0]
                pos_y = i * char_size[1]
                
                # Paste the image into the final image
                final_image.paste(img, (pos_x, pos_y))

    # Resize the final image to the desired output size
    final_image = final_image.resize(output_size)
    
    return final_image

# Example usage:
ascii_art = [
    [' ', ' ', 'G'],
    ['$', '$', '$'],
    [' ', '$', '$']
]

# Map characters to image file paths
img_map = {
    'G': 'img/cave-32.png',
    ' ': 'img/empty-32.png',
    '$': 'img/wall-32.png',
    'c': 'img/treasure-32.png',
    ':': 'img/scroll-32.png',
}

final_image = create_image_from_ascii(square_data_array_2, img_map, output_size=(2500, 2500), char_size=(50, 50)) # Create the final image from ASCII art and image map
final_image.save('output_image.png') # Save the image